Setup & imports

In [60]:
import re
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer  # or the specific tokenizer you use

In [61]:
# Reading CSV file using pandas
# Basic usage:
df = pd.read_csv(r"D:\UT_Austin\sem4\physionet.org\files\labelled-notes-hospital-course\1.1.0\mimic-iv-bhc.csv")

print("CSV file loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()  # Display first 5 rows

CSV file loaded successfully!
Shape: (270033, 5)
Columns: ['note_id', 'input', 'target', 'input_tokens', 'target_tokens']


,note_id,input,target,input_tokens,target_tokens
0,10000032-DS-21,<SEX> F <SERVICE> MEDICINE <ALLERGIES> No Know...,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1946,231
1,10000032-DS-22,<SEX> F <SERVICE> MEDICINE <ALLERGIES> Percoce...,"___ with HIV on HAART, HCV cirrhosis with asci...",2183,810
2,10000117-DS-21,<SEX> F <SERVICE> MEDICINE <ALLERGIES> omepraz...,Ms. ___ is a ___ with history of GERD who pres...,1060,172
3,10000117-DS-22,<SEX> F <SERVICE> ORTHOPAEDICS <ALLERGIES> ome...,The patient presented to the emergency departm...,1195,330
4,10000248-DS-10,<SEX> M <SERVICE> MEDICINE <ALLERGIES> No Know...,Mr. ___ is a ___ with history of mild FVIII de...,1961,230


Basic cleaning / normalization functions

In [62]:
def normalize_whitespace(text: str) -> str:
    # collapse multiple spaces / newlines into single
    text = re.sub(r'\r', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def clean_placeholders(text: str) -> str:
    # e.g. ensure consistent casing / spacing around placeholders
    # This is generic; adapt if you have more placeholder variants
    text = re.sub(r'<\s*SEX\s*>', '<SEX>', text, flags=re.IGNORECASE)
    text = re.sub(r'<\s*SERVICE\s*>', '<SERVICE>', text, flags=re.IGNORECASE)
    text = re.sub(r'<\s*ALLERGIES\s*>', '<ALLERGIES>', text, flags=re.IGNORECASE)
    return text

def remove_control_characters(text: str) -> str:
    # remove non-printable / weird control chars
    # e.g. anything in category Cc except \n
    cleaned = ''.join(ch for ch in text if (ch == '\n' or (ord(ch) >= 32 and ord(ch) != 127)))
    return cleaned

def preprocess_text(text: str) -> str:
    if text is None:
        return ""
    text = normalize_whitespace(text)
    text = clean_placeholders(text)
    text = remove_control_characters(text)
    return text


In [63]:
df['input_clean'] = df['input'].map(preprocess_text)
df['target_clean'] = df['target'].map(preprocess_text)


Filtering / dropping low-quality / edge examples

In [64]:
# Drop examples where input or target is empty / very short
MIN_INPUT_TOKENS = 50
MIN_TARGET_TOKENS = 20

mask_good = (df['input_clean'].map(len) > MIN_INPUT_TOKENS) & \
            (df['target_clean'].map(len) > MIN_TARGET_TOKENS)

df = df[mask_good].copy()

# Optionally: drop outliers where target is longer than input (if that happens)
df = df[df['target_tokens'] < df['input_tokens'] * 1.2]  # example threshold


Tokenization & truncation / handling long inputs

In [65]:
tokenizer = AutoTokenizer.from_pretrained("Falconsai/medical_summarization")  # e.g. “t5-base”, “llama-…”, etc.
# You'll also need the model itself
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("Falconsai/medical_summarization")
MAX_INPUT_LEN = 2048  # adjust depending on your model capacity
MAX_TARGET_LEN = 512  # adjust

def encode_example(input_text: str, target_text: str):
    # encode input and target, with truncation if too long
    enc = tokenizer(
        input_text,
        truncation=True,
        max_length=MAX_INPUT_LEN,
        padding=False,
        return_tensors="pt",
    )
    dec = tokenizer(
        target_text,
        truncation=True,
        max_length=MAX_TARGET_LEN,
        padding=False,
        return_tensors="pt",
    )
    return {
        'input_ids': enc.input_ids.squeeze(0),
        'attention_mask': enc.attention_mask.squeeze(0),
        'labels': dec.input_ids.squeeze(0)
    }

# Example application
encoded = encode_example(df.iloc[0]['input_clean'], df.iloc[0]['target_clean'])
print(f"Setup successful! Input shape: {encoded['input_ids'].shape}, Labels shape: {encoded['labels'].shape}")


c:\Users\ajays\anaconda3\envs\py311_cuda_env\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Setup successful! Input shape: torch.Size([2048]), Labels shape: torch.Size([285])


Creating dataset variants (full, section-filtered, chunked)

4.1 Section filtering

In [66]:
# Example: extract “HOSPITAL COURSE” section if present
def extract_section(text: str, section_header: str) -> str:
    # naive: split by the header and then stop when next header appears
    # This assumes section headers in uppercase and followed by colon, e.g. “HOSPITAL COURSE:”
    pattern = rf"{section_header}\s*:"
    parts = re.split(pattern, text, flags=re.IGNORECASE)
    if len(parts) <= 1:
        return ""  # section not found
    rest = parts[1]
    # Now stop when you see another section header (e.g. all caps + colon)
    # e.g. next_header = uppercase letters + colon
    m = re.search(r"\n[A-Z ]{3,50}:\s", rest)
    if m:
        rest = rest[: m.start()]
    return rest.strip()

# Example: create version of input using only “HOSPITAL COURSE” section
df['section_hcourse'] = df['input_clean'].map(lambda x: extract_section(x, "HOSPITAL COURSE"))

# Use that as an alternative input version if not empty, else fallback to full
df['input_section_or_full'] = df.apply(lambda row: row['section_hcourse'] if len(row['section_hcourse']) > 0 else row['input_clean'], axis=1)


4.2 Chunking / sliding windows

In [67]:
def chunk_input(input_ids: torch.Tensor, attention_mask: torch.Tensor, chunk_size: int, stride: int = None):
    # Returns list of (chunk_input_ids, chunk_masks)
    if stride is None:
        stride = chunk_size  # non overlapping
    chunks = []
    L = input_ids.size(0)
    start = 0
    while start < L:
        end = min(start + chunk_size, L)
        chunk_ids = input_ids[start:end]
        chunk_mask = attention_mask[start:end]
        chunks.append((chunk_ids, chunk_mask))
        if end == L:
            break
        start += stride
    return chunks

# Example: for each example, produce chunks
CHUNK_SIZE = 1024
STRIDE = 512

row = encoded  # from earlier encode_example
input_ids = row['input_ids']
mask = row['attention_mask']
chunks = chunk_input(input_ids, mask, CHUNK_SIZE, STRIDE)
print(f"Number of chunks: {len(chunks)}")


Number of chunks: 3


Custom Dataset & DataLoader

In [68]:
class BHC_Dataset(Dataset):
    def __init__(self, df, tokenizer, max_input_len=2048, max_target_len=512, variant="full"):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len
        self.variant = variant  # e.g. "full", "section_filtered", etc.

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        if self.variant == "section_filtered":
            input_text = row.get('input_section_or_full', row['input_clean'])
        else:
            input_text = row['input_clean']
        target_text = row['target_clean']
        enc = self.tokenizer(
            input_text,
            truncation=True,
            max_length=self.max_input_len,
            padding=False,
            return_tensors="pt",
        )
        dec = self.tokenizer(
            target_text,
            truncation=True,
            max_length=self.max_target_len,
            padding=False,
            return_tensors="pt",
        )
        item = {
            'input_ids': enc.input_ids.squeeze(0),
            'attention_mask': enc.attention_mask.squeeze(0),
            'labels': dec.input_ids.squeeze(0)
        }
        return item

# Example usage
ds = BHC_Dataset(df, tokenizer, max_input_len=2048, max_target_len=512, variant="full")
dl = DataLoader(ds, batch_size=4, shuffle=True, collate_fn=lambda x: x)


Logging / Documentation

In [69]:
def log_stats(df_before: pd.DataFrame, df_after: pd.DataFrame, step_name: str):
    print(f"Step: {step_name}")
    print(f"Before: {len(df_before)} examples")
    print(f"After: {len(df_after)} examples")
    dropped = len(df_before) - len(df_after)
    print(f"Dropped: {dropped} examples ({100 * dropped / len(df_before):.1f}%)")


In [70]:
df0 = df.copy()
# filtering step
df = df[mask_good].copy()
log_stats(df0, df, "drop short / empty examples")


Step: drop short / empty examples
Before: 269059 examples
After: 269059 examples
Dropped: 0 examples (0.0%)


C:\Users\ajays\AppData\Local\Temp\ipykernel_21684\2926529693.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[mask_good].copy()


In [71]:
df.to_csv("bhc_preprocessed_v1.csv", index=False)

In [72]:
def summarize(text, max_input_length=1024, max_summary_length=256):
    inputs = tokenizer(text, return_tensors="pt",
                       truncation=True, max_length=max_input_length, 
                       padding=True)  # or pad to a min batch size
    output = model.generate(**inputs, max_length=max_summary_length, 
                            num_beams=2, early_stopping=True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary

In [73]:
print(df.iloc[0]['input_clean'])

<SEX> F <SERVICE> MEDICINE <ALLERGIES> No Known Allergies / Adverse Drug Reactions <ATTENDING> ___ <CHIEF COMPLAINT> Worsening ABD distension and pain <MAJOR SURGICAL OR INVASIVE PROCEDURE> Paracentesis <HISTORY OF PRESENT ILLNESS> ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week. Pt reports self-discontinuing lasix and spirnolactone ___ weeks ago, because she feels like "they don't do anything" and that she "doesn't want to put more chemicals in her." She does not follow Na-restricted diets. In the past week, she notes that she has been having worsening abd distension and discomfort. She denies ___ edema, or SOB, or orthopnea. She denies f/c/n/v, d/c, dysuria. She had food poisoning a week ago from eating stale cake (n/v 20 min after food ingestion), which resolved the same day. She denies other recent illness or sick contacts. She notes that she has been noticing gum bleeding while brushing he

In [74]:
# Example
instruction = (
    "Summarize the following hospital discharge note into a concise, factual "
    "Brief Hospital Course summary:\n\n"
)
summary = summarize(instruction + df.iloc[0]['input_clean'], max_input_length=1024, max_summary_length=300)
print(summary)

SEX> SERVICE> MEDICINE ALLERGIES> No Known Allergies / Adverse Drug Reactions ATTENDING> ___ CHIEF COMPLAINT> Worsening abd distension and pain MAJOR SURGICAL OR INVASIVE PROCEDURE> Paracentesis HISTORY OF PRESENT ILLNESS> ___ HCV cirrhosis c/b ascites, hiv on ART, h/o IVDU, COPD, bioplar, PTSD, presented from OSH ED with worsening abd distension over past week.


Train / Validation Split

In [75]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
print(f"Original Training length: {len(train_df)} | Original Validation Length: {len(val_df)}")


Original Training length: 242153 | Original Validation Length: 26906


sample the dataset

In [76]:
# train_df = train_df.sample(2000, random_state=42)
# val_df = val_df.sample(500, random_state=42)
# print(f"Sampled Training length: {len(train_df)} | Sampled Validation Length: {len(val_df)}")

In [77]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Falconsai/medical_summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # This is PyTorch by default


c:\Users\ajays\anaconda3\envs\py311_cuda_env\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [78]:
from torch.utils.data import Dataset, DataLoader

class SummarizationDataset(Dataset):
    def __init__(self, df, tokenizer, max_input_len=1024, max_target_len=256):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        inputs = tokenizer(
            row["input_clean"],
            truncation=True,
            padding="max_length",
            max_length=self.max_input_len,
            return_tensors="pt",
        )
        targets = tokenizer(
            row["target_clean"],
            truncation=True,
            padding="max_length",
            max_length=self.max_target_len,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs.input_ids.squeeze(),
            "attention_mask": inputs.attention_mask.squeeze(),
            "labels": targets.input_ids.squeeze(),
        }

train_dataset = SummarizationDataset(train_df, tokenizer)
val_dataset = SummarizationDataset(val_df, tokenizer)
model.gradient_checkpointing_enable()

Fine-Tuning

LORA config adapter

In [79]:
# Enable optimizations
torch.backends.cudnn.benchmark = True     # Good for fixed input sizes
torch.backends.cuda.matmul.allow_tf32 = True  # Good for newer GPUs
# Safe GPU optimizations
try:
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True
        if hasattr(torch.backends.cuda.matmul, 'allow_tf32'):
            torch.backends.cuda.matmul.allow_tf32 = True
        print("✅ GPU optimizations enabled")
    else:
        print("⚠️ CUDA not available, skipping GPU optimizations")
except Exception as e:
    print(f"⚠️ Could not enable optimizations: {e}")

✅ GPU optimizations enabled


In [80]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)
model = get_peft_model(model, lora_config)

In [81]:
from transformers import Trainer, TrainingArguments, TrainerCallback
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    warmup_steps=100,                      # 🔥 Add warmup for stability
    # max_steps=300,                     # 🔥 Move max_steps here
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,                     # 🔥 enable mixed precision
    save_total_limit=3,
    report_to="none",
    dataloader_num_workers=0,      # fewer threads on Windows
    logging_steps=200,
    resume_from_checkpoint=True,  # 🔥 Add this
    dataloader_pin_memory=False,           # 🔥 Reduce memory usage
    remove_unused_columns=True,            # 🔥 Clean up unused data
    gradient_checkpointing=True,           # 🔥 Trade compute for memory
    optim="adamw_torch",  # 🔥 Use non-deprecated AdamW

)
# Enhanced callback for longer training
class EnhancedCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            # Add timestamp and step info
            import datetime
            timestamp = datetime.datetime.now().strftime("%H:%M:%S")
            if 'loss' in logs:
                print(f"[{timestamp}] Step {state.global_step}: Loss = {logs['loss']:.4f}")
            if 'eval_loss' in logs:
                print(f"[{timestamp}] Step {state.global_step}: Eval Loss = {logs['eval_loss']:.4f}")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[EnhancedCallback()],
)
# -------------------------------
# 8. Train (start small)
# -------------------------------
trainer.train()

  0%|          | 0/15134 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\ajays\anaconda3\envs\py311_cuda_env\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


[08:52:27] Step 200: Loss = 4.6693
{'loss': 4.6693, 'learning_rate': 4.967407210323268e-05, 'epoch': 0.01}
[08:58:08] Step 400: Loss = 3.9221
{'loss': 3.9221, 'learning_rate': 4.9008913130238126e-05, 'epoch': 0.03}
[09:03:40] Step 600: Loss = 3.7591
{'loss': 3.7591, 'learning_rate': 4.834375415724358e-05, 'epoch': 0.04}
[09:09:13] Step 800: Loss = 3.6854
{'loss': 3.6854, 'learning_rate': 4.768192097911401e-05, 'epoch': 0.05}
[09:14:51] Step 1000: Loss = 3.6661
{'loss': 3.6661, 'learning_rate': 4.7016762006119464e-05, 'epoch': 0.07}


  0%|          | 0/6727 [00:00<?, ?it/s]

[09:27:59] Step 1000: Eval Loss = 3.4089
{'eval_loss': 3.4088950157165527, 'eval_runtime': 787.5448, 'eval_samples_per_second': 34.164, 'eval_steps_per_second': 8.542, 'epoch': 0.07}


...

  0%|          | 0/6727 [00:00<?, ?it/s]

[19:02:12] Step 15000: Eval Loss = 3.0827
{'eval_loss': 3.0826780796051025, 'eval_runtime': 783.5047, 'eval_samples_per_second': 34.341, 'eval_steps_per_second': 8.586, 'epoch': 0.99}


c:\Users\ajays\anaconda3\envs\py311_cuda_env\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


{'train_runtime': 37175.6398, 'train_samples_per_second': 6.514, 'train_steps_per_second': 0.407, 'train_loss': 3.4158053352131117, 'epoch': 1.0}


TrainOutput(global_step=15134, training_loss=3.4158053352131117, metrics={'train_runtime': 37175.6398, 'train_samples_per_second': 6.514, 'train_steps_per_second': 0.407, 'train_loss': 3.4158053352131117, 'epoch': 1.0})

In [82]:
model.save_pretrained("./medical_summarizer_lora_v1")
tokenizer.save_pretrained("./medical_summarizer_lora_v1")
print("✅ Training complete and model saved.")

✅ Training complete and model saved.


EVALUATION: rouge or bleu
Common metrics available: "bleu", "rouge", "meteor", "bertscore", "exact_match", etc.

In [83]:
import evaluate
from tqdm import tqdm

metric = evaluate.load("rouge")

def evaluate_model(dataset, num_samples=50):
    model.eval()
    preds, refs = [], []
    for i in tqdm(range(num_samples)):
        text = dataset.df.iloc[i]["input_clean"]
        reference = dataset.df.iloc[i]["target_clean"]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
        with torch.no_grad():
            output = model.generate(**inputs, max_new_tokens=256)
        pred = tokenizer.decode(output[0], skip_special_tokens=True)
        preds.append(pred)
        refs.append(reference)
    return metric.compute(predictions=preds, references=refs)

results = evaluate_model(val_dataset, num_samples=30)
print(results)


100%|██████████| 30/30 [02:25<00:00,  4.84s/it]


{'rouge1': 0.24322987122995235, 'rouge2': 0.061644801138736, 'rougeL': 0.15954732842055513, 'rougeLsum': 0.15995710914879385}
